In [2]:
from numpy import genfromtxt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import argparse
import codecs
import numpy as np
import os
import shutil
import torch
import yaml
from datetime import datetime
from torch.utils.data.sampler import RandomSampler
from torch.autograd import Variable
EMBEDDING_SIZE = 50
HEADLIN_SIZE = 50
import scipy

In [3]:
data = pd.read_csv('Combined_News_DJIA.csv')

In [4]:
train = data.iloc[0:1611]
test = data[1611:]
print(train.shape)
print(test.shape)

(1611, 27)
(378, 27)


In [5]:
trainheadlines = data.iloc[0:1611, 2:27].values
trainLabels = data.iloc[0:1611, 1].values
testheadlines = data.iloc[1611:, 2:27].values
testLabels = data.iloc[1611:, 1].values

print(trainheadlines.shape)
print(trainLabels[:10])

(1611, 25)
[0 1 0 0 1 1 0 0 1 1]


In [6]:
trainheadlines = []
trainLabels = []
trainRawheadlines = []
for row in range(0,len(train.index)):
    trainRawheadlines += [str(x) for x in train.iloc[row,2:27]]
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))
    trainLabels.append(int(train.iloc[row,1]))

basicvectorizer = CountVectorizer(min_df=3)
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

testheadlines = []
testLabels = []
testRawheadlines = []
for row in range(0,len(test.index)):
    testRawheadlines += [str(x) for x in test.iloc[row,2:27]]
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
    testLabels.append(test.iloc[row,1])

basictest = basicvectorizer.transform(testheadlines)
vocab_size = len(basicvectorizer.vocabulary_.items())
embeddings_index = {}

(1611, 14073)


In [7]:
minLen = 500
hist = {}
for i,h in enumerate(trainRawheadlines):
    l = len(basicvectorizer.build_tokenizer()(h))
    if l in hist:
        hist[l] += 1
    else:
        hist[l] = 1
    if l < minLen:
        minLen = l
        print(i, h, minLen)
print(hist)

0 b"Georgia 'downs two Russian warplanes' as countries move to brink of war" 12
1 b'BREAKING: Musharraf to be impeached.' 5
51 b"Russia 'ends Georgia operation'" 4
169 b"NATO's Hour" 2
6947 nan 1
{1: 7, 2: 10, 3: 127, 4: 393, 5: 1097, 6: 1910, 7: 2617, 8: 3046, 9: 2995, 10: 2809, 11: 2276, 12: 1861, 13: 1480, 14: 1349, 15: 1139, 16: 1079, 17: 994, 18: 906, 19: 809, 20: 836, 21: 795, 22: 737, 23: 773, 24: 755, 25: 670, 26: 633, 27: 584, 28: 596, 29: 566, 30: 550, 31: 513, 32: 448, 33: 471, 34: 386, 35: 384, 36: 345, 37: 310, 38: 294, 39: 323, 40: 292, 41: 296, 42: 278, 43: 269, 44: 230, 45: 195, 46: 201, 47: 198, 48: 129, 49: 110, 50: 82, 51: 52, 52: 37, 53: 11, 54: 8, 55: 5, 56: 7, 58: 1, 59: 1}


In [8]:
# print(trainheadlines[0])
# print(basictrain[0])
basicvectorizer.inverse_transform([0])
print(basicvectorizer.inverse_transform(basictrain[0]))
cx = scipy.sparse.coo_matrix(basictrain[0])

for i,j,v in zip(cx.row, cx.col, cx.data):
    print("(%d, %d), %s " % (i,j,v))
    

[array(['surge', 'kidnapping', 'mexico', 'banned', 'illnesses', 'mental',
       'suffering', 'visitors', 'work', 'your', 'you', 'series', 'again',
       'shoe', 'indian', 'crisis', 'caucasus', 'report', 'amp', 'news',
       'idea', 'bad', 'attacking', 'thinks', 'pentagon', 'why', 'region',
       'breakaway', 'forces', 'off', 'fight', 'help', 'iraq', 'soldiers',
       '000', 'withdraw', 'programme', 'nuclear', 'its', 'protest',
       'sanctions', 'new', 'approved', 'union', 'european', 'day', 'busy',
       'hostilities', 'case', 'in', 'victory', 'prepared', 'barak',
       'ehud', 'minister', 'defense', 'strike', 'an', 'prevent', 'act',
       'not', 'us', 'condoleezza', 'backlash', 'islamist', 'faces',
       'qaeda', 'al', 'scale', 'full', 'unleash', 'nato', 'involved',
       'gets', 'if', 'start', 'iii', 'world', 'affairs', 'other', 'out',
       'stay', 'bush', 'tells', 'china', 'journalism', 'decline', 'proof',
       'further', 'yet', 'disgrace', 'fucking', 'what', 'olympi

In [9]:
embeddings_index_300 = {}
embeddings_index = {}
def load_embeddings(BASE_DIR = 'C:/Users/rachu/Google Drive/UCI/q3/Independent Study'):
    with open('glove.6B.50d.txt', encoding = 'utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Found %s word vectors.' % len(embeddings_index))
load_embeddings()


Found 400000 word vectors.


In [10]:
print(embeddings_index['misdoings'])
embeddings_index['hi']-embeddings_index['hello']

[ 0.38901   -0.52625    0.14977    0.051708  -0.89573   -0.19634
  0.49229    0.3524    -0.19235    0.20916   -0.20043    0.8726
 -0.44704   -0.064333   0.21732   -0.35044   -0.4289    -0.50382
  1.0496     0.42719    0.089963  -0.10629   -0.077512  -0.24051
  0.14269    0.85873   -0.48103   -0.44698    0.27143    0.21746
 -1.3416     0.64947    0.85329   -0.33107   -0.87683    0.78465
 -0.7616    -0.57617   -0.12419    0.08886   -0.0070094 -0.47746
  0.17203    0.77937    0.45732   -0.76784    0.064205   0.90093
  0.39476   -0.42696  ]


array([-1.5815997e-01, -4.5665002e-01,  2.0714401e-01,  1.3322500e+00,
       -1.1374409e+00, -9.2688000e-01,  1.0003400e+00, -4.4798002e-01,
        3.3779979e-02,  2.9936004e-01,  6.1003304e-01, -9.0547180e-01,
       -9.3536997e-01, -5.3293896e-01, -6.5013295e-01,  1.1399100e+00,
        9.4141495e-01, -3.8995999e-01,  3.1202000e-01,  6.0972500e-01,
        4.0626001e-01, -6.8052995e-01, -5.4380298e-01,  7.0419997e-02,
       -1.2301461e+00,  1.2548000e-01, -4.8060000e-02, -9.7613001e-01,
       -8.1930012e-02,  2.2979999e-01,  6.6057998e-01,  4.4749001e-01,
        8.1035995e-01,  9.5780015e-02,  5.8307999e-01, -2.8990996e-01,
       -6.2957501e-01,  1.5706100e+00,  3.7344301e-01,  4.6988997e-01,
       -1.1308999e-01, -6.6519976e-02,  1.6599200e+00,  8.9997053e-04,
        9.9190998e-01,  1.1799999e-02,  2.2280100e-01,  1.0599971e-02,
        3.8792700e-01,  3.0888999e-01], dtype=float32)

In [11]:
embedding_matrix_head = np.random.rand(len(basicvectorizer.vocabulary_.items()), EMBEDDING_SIZE)
for word, i in basicvectorizer.vocabulary_.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_head[i] = embedding_vector

In [12]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf
import math

class CNNLayer(nn.Module):
    def __init__(self, input_dim):
        super(CNNLayer, self).__init__()
        self.input_dim = input_dim
        self.kernel_size = 4
        self.cnn = nn.Conv2d(1, 1, self.kernel_size)
        self.fc1 = nn.Linear(input_dim - self.kernel_size + 1, 50)
        self.fc2 = nn.Linear(50, 2)
        self.cnnDropout = nn.Dropout2d(0.3)
        self.fc1Dropout = nn.Dropout2d(0.2)
        self.fc2Dropout = nn.Dropout2d(0.2)
        self.embedding = nn.Embedding(vocab_size, EMBEDDING_SIZE)
        self.embedding.weight = nn.Parameter(torch.from_numpy(embedding_matrix_head))
#         self.embedding.weight.requires_grad = False

    def forward(self, headlines):
        embeddingSum = [float('-inf')]*self.input_dim
        cnnForInput = None
        for i, headline in enumerate(headlines):
            words = basicvectorizer.build_tokenizer()(headline)
#             embeddingHeadline = np.zeros((len(words), EMBEDDING_SIZE))
            embeddingHeadline = None
            if len(words) < self.kernel_size:
                continue
            for i, word in enumerate(words):
                word = word.lower()
                if word in basicvectorizer.vocabulary_:
                    embedOut = self.embedding(Variable(torch.LongTensor([int(basicvectorizer.vocabulary_[word])])).cuda())
                    if embeddingHeadline is None:
                        embeddingHeadline = embedOut
                    else:
                        embeddingHeadline = torch.cat((embeddingHeadline, embedOut), 0)
#             embeddingHeadlineVar = Variable(torch.from_numpy(embeddingHeadline)).view(1, 1, -1, EMBEDDING_SIZE).float().cuda()
            embeddingHeadlineVar = embeddingHeadline.view(1, 1, -1, EMBEDDING_SIZE).float().cuda()
#             print(len(words), embeddingHeadlineVar.size(2))
            if embeddingHeadlineVar.size(2) < self.kernel_size:
                continue
            cnnHeadline = self.cnnDropout(self.cnn(embeddingHeadlineVar))
            cnnHeadlineSum = torch.sum(cnnHeadline, 1)
            if cnnForInput is None:
                cnnForInput = cnnHeadlineSum
            else:
                cnnForInput = torch.cat((cnnForInput, cnnHeadlineSum), 1)
                
        cnnForInputMaxed = torch.max(cnnForInput, 1)[0].view(1, 1, -1)
        fcOutput = self.fc2Dropout(self.fc2(self.fc1Dropout(self.fc1(cnnForInputMaxed))))
        log_probs = F.log_softmax(fcOutput[0], dim=-1)
        return log_probs


C:\Users\rachu\AppData\Local\conda\conda\envs\tf-cpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
# Correct one

import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf
import math

class CNNLayer(nn.Module):
    def __init__(self, input_dim):
        super(CNNLayer, self).__init__()
        self.input_dim = input_dim
        self.kernel_size = (4, EMBEDDING_SIZE)
        self.cnn = nn.Conv2d(1, 1, self.kernel_size)
        self.fc1 = nn.Linear(input_dim, 50)
        self.fc2 = nn.Linear(50, 2)
        self.cnnDropout = nn.Dropout2d(0.3)
        self.fc1Dropout = nn.Dropout2d(0.2)
        self.fc2Dropout = nn.Dropout2d(0.2)

    def forward(self, headlines):
        embeddingSum = [float('-inf')]*self.input_dim
        cnnForInput = None
        for i, headline in enumerate(headlines):
            words = basicvectorizer.build_tokenizer()(headline)
            embeddingHeadline = np.zeros((len(words), EMBEDDING_SIZE))
            for i, word in enumerate(words):
                word = word.lower()
                if word in embeddings_index:
                    embeddingHeadline[i] = embeddings_index[word]
            embeddingHeadlineVar = Variable(torch.from_numpy(embeddingHeadline)).view(1, 1, -1, EMBEDDING_SIZE).float().cuda()
            if embeddingHeadlineVar.size(2) < self.kernel_size[0]:
                cnnHeadline = Variable(torch.zeros(1, 1, 10, 1)).cuda()
            else:
                cnnHeadline = self.cnnDropout(self.cnn(embeddingHeadlineVar))
            
            cnnHeadlineMax = torch.max(cnnHeadline, 2)[0]
            if cnnForInput is None:
                cnnForInput = cnnHeadlineMax
            else:
                cnnForInput = torch.cat((cnnForInput, cnnHeadlineMax), 1)
            print(embeddingHeadlineVar.size(), cnnHeadline.size(), cnnHeadlineMax.size())

        cnnForInput = cnnForInput.view(1, 1, -1)
        print(cnnForInput.size())
        fcOutput = self.fc2Dropout(self.fc2(self.fc1Dropout(self.fc1(cnnForInput))))
        log_probs = F.log_softmax(fcOutput[0], dim=-1)
        return log_probs


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf
import math

class RConvLayer(nn.Module):
    def __init__(self, input_dim):
        super(CNNLayer, self).__init__()
        self.input_dim = input_dim
        self.kernel_size = (4, EMBEDDING_SIZE)
        self.cnn = nn.Conv2d(1, 1, self.kernel_size)
        self.cnnDropout = nn.Dropout2d(0.3)

    def forward(self, headlines):
        cnnForInput = None
        for i, headline in enumerate(headlines):
            words = basicvectorizer.build_tokenizer()(headline)
            embeddingHeadline = np.zeros((len(words), EMBEDDING_SIZE))
            for i, word in enumerate(words):
                word = word.lower()
                if word in embeddings_index:
                    embeddingHeadline[i] = embeddings_index[word]
            embeddingHeadlineVar = Variable(torch.from_numpy(embeddingHeadline)).view(1, 1, -1, EMBEDDING_SIZE).float().cuda()
            if embeddingHeadlineVar.size(2) < self.kernel_size[0]:
                cnnHeadline = Variable(torch.zeros(1, 1, 10, 1)).cuda()
            else:
                cnnHeadline = self.cnnDropout(self.cnn(embeddingHeadlineVar))
            
            cnnHeadlineMax = torch.max(cnnHeadline, 2)[0]
            if cnnForInput is None:
                cnnForInput = cnnHeadlineMax
            else:
                cnnForInput = torch.cat((cnnForInput, cnnHeadlineMax), 1)
            print(embeddingHeadlineVar.size(), cnnHeadline.size(), cnnHeadlineMax.size())

        cnnForInput = cnnForInput.view(1, 1, -1)
        return cnnForInput


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf
import math

class ForwardLayer(nn.Module):
    def __init__(self, input_dim):
        self.fc1 = nn.Linear(input_dim, 2)
        self.fc1Dropout = nn.Dropout2d(0.2)
        self.convLayer = RConvLayer(input_dim)
        self.criterion = torch.nn.NLLLoss()

    def forward(self, train_features, train_labels):
        total_loss = 0
        trainLabelVar = Variable(torch.LongTensor(np.asarray(train_labels))).cuda()
        train_loss = []
        train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
        for i in range(len(train_labels)):
            if len(train_features[i][0]) < 25:
                continue
            convOutput = self.convLayer(train_features[i][0][-25:])
            fcOutput = self.fc1Dropout(self.fc1(convOutput))
            log_probs = F.log_softmax(fcOutput[0], dim=-1)
            loss = self.criterion(log_probs, trainLabelVar[i])
            total_loss += loss
        train_loss.append(total_loss.data)
        print(train_log_string % (datetime.now(), epoch, i+1, len(train_labels), np.mean(train_loss)))
        return total_loss

In [14]:
# Correct one

cnnLayer = CNNLayer(25).cuda()
learning_rate = 0.3
cnnOptimizer = torch.optim.Adam(cnnLayer.parameters(), lr=learning_rate)
epoch = 0
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
train_loss = []
criterion = torch.nn.NLLLoss()
trainLabelVar = Variable(torch.LongTensor(trainLabels)).cuda()
cell = None
hidden = None
while epoch < 200:
    
    total_loss = 0
    for i in range(len(trainLabels)):
        cnnOptimizer.zero_grad()
        old_hidden = hidden
        log_probs = cnnLayer(trainRawheadlines[i*25:(i+1)*25])
        loss = criterion(log_probs, trainLabelVar[i])
        total_loss += loss
    total_loss.backward()
    cnnOptimizer.step()
    train_loss.append(total_loss.data)
    if ((epoch) % 1) == 0:
        print(train_log_string % (datetime.now(), epoch, i+1, len(trainheadlines), np.mean(train_loss)))
    epoch = epoch + 1

torch.Size([1, 1, 12, 50]) torch.Size([1, 1, 9, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 5, 50]) torch.Size([1, 1, 2, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 13, 50]) torch.Size([1, 1, 10, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 20, 50]) torch.Size([1, 1, 17, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 19, 50]) torch.Size([1, 1, 16, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 14, 50]) torch.Size([1, 1, 11, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 13, 50]) torch.Size([1, 1, 10, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 27, 50]) torch.Size([1, 1, 24, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 13, 50]) torch.Size([1, 1, 10, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 17, 50]) torch.Size([1, 1, 14, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 12, 50]) torch.Size([1, 1, 9, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 30, 50]) torch.Size([1, 1, 27, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 50]) torc

torch.Size([1, 1, 12, 50]) torch.Size([1, 1, 9, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 9, 50]) torch.Size([1, 1, 6, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 13, 50]) torch.Size([1, 1, 10, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 7, 50]) torch.Size([1, 1, 4, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 25])
torch.Size([1, 1, 17, 50]) torch.Size([1, 1, 14, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 50]) torch.Size([1, 1, 7, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 5, 50]) torch.Size([1, 1, 2, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 5, 50]) torch.Size([1, 1, 2, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 30, 50]) torch.Size([1, 1, 27, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 7, 50]) torch.Size([1, 1, 4, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 15, 50]) torch.Size([1, 1, 12, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 11, 50]) torch.Size([1, 1, 8, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 25, 50]) torch.Size([1, 1, 22, 1]) torch.Size([1, 1, 1])
torch.Size([1, 

torch.Size([1, 1, 32, 50]) torch.Size([1, 1, 29, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 50]) torch.Size([1, 1, 7, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 6, 50]) torch.Size([1, 1, 3, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 12, 50]) torch.Size([1, 1, 9, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 29, 50]) torch.Size([1, 1, 26, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 50]) torch.Size([1, 1, 7, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 25])
torch.Size([1, 1, 11, 50]) torch.Size([1, 1, 8, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 7, 50]) torch.Size([1, 1, 4, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 23, 50]) torch.Size([1, 1, 20, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 24, 50]) torch.Size([1, 1, 21, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1

torch.Size([1, 1, 10, 50]) torch.Size([1, 1, 7, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 9, 50]) torch.Size([1, 1, 6, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 14, 50]) torch.Size([1, 1, 11, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 9, 50]) torch.Size([1, 1, 6, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 12, 50]) torch.Size([1, 1, 9, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 9, 50]) torch.Size([1, 1, 6, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 11, 50]) torch.Size([1, 1, 8, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 50]) torch.Size([1, 1, 7, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 23, 50]) torch.Size([1, 1, 20, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 26, 50]) torch.Size([1, 1, 23, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 9, 50]) torch.Size([1, 1, 6, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 12, 50]) torch.Size([1, 1, 9, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 9, 50]) torch.Size([1,

torch.Size([1, 1, 10, 50]) torch.Size([1, 1, 7, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 7, 50]) torch.Size([1, 1, 4, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 14, 50]) torch.Size([1, 1, 11, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 9, 50]) torch.Size([1, 1, 6, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 11, 50]) torch.Size([1, 1, 8, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 11, 50]) torch.Size([1, 1, 8, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 7, 50]) torch.Size([1, 1, 4, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 6, 50]) torch.Size([1, 1, 3, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 50]) torch.Size([1, 1, 7, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 6, 50]) torch.Size([1, 1, 3, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 7, 50]) torch.Size([1, 1, 4, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 7, 50]) torch.Size([1, 1, 4, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 25])
torch.Size([1, 1, 5, 50]) torch.Size([1, 1, 2, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 20, 

...

torch.Size([1, 1, 40, 50]) torch.Size([1, 1, 37, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 34, 50]) torch.Size([1, 1, 31, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 20, 50]) torch.Size([1, 1, 17, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 24, 50]) torch.Size([1, 1, 21, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 21, 50]) torch.Size([1, 1, 18, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 34, 50]) torch.Size([1, 1, 31, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 18, 50]) torch.Size([1, 1, 15, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 33, 50]) torch.Size([1, 1, 30, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 6, 50]) torch.Size([1, 1, 3, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 24, 50]) torch.Size([1, 1, 21, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 12, 50]) torch.Size([1, 1, 9, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 28, 50]) torch.Size([1, 1, 25, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 17, 50]) torch.Size([1, 1, 14, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 4, 50]) to

torch.Size([1, 1, 11, 50]) torch.Size([1, 1, 8, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 16, 50]) torch.Size([1, 1, 13, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 50]) torch.Size([1, 1, 7, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 9, 50]) torch.Size([1, 1, 6, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 30, 50]) torch.Size([1, 1, 27, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 23, 50]) torch.Size([1, 1, 20, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 37, 50]) torch.Size([1, 1, 34, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 14, 50]) torch.Size([1, 1, 11, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 43, 50]) torch.Size([1, 1, 40, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 36, 50]) torch.Size([1, 1, 33, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 15, 50]) torch.Size([1, 1, 12, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 25])
torch.Size([1, 1, 15, 50]) torch.Size([1, 1, 12, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 43, 50]) torch.Size([1, 1, 40, 1]) torch.Size([1, 1, 1])
torch.

torch.Size([1, 1, 22, 50]) torch.Size([1, 1, 19, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 11, 50]) torch.Size([1, 1, 8, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 7, 50]) torch.Size([1, 1, 4, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 33, 50]) torch.Size([1, 1, 30, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 11, 50]) torch.Size([1, 1, 8, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 15, 50]) torch.Size([1, 1, 12, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 6, 50]) torch.Size([1, 1, 3, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 23, 50]) torch.Size([1, 1, 20, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 26, 50]) torch.Size([1, 1, 23, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 6, 50]) torch.Size([1, 1, 3, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 29, 50]) torch.Size([1, 1, 26, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 25])
torch.Size([1,

torch.Size([1, 1, 6, 50]) torch.Size([1, 1, 3, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 23, 50]) torch.Size([1, 1, 20, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 11, 50]) torch.Size([1, 1, 8, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 23, 50]) torch.Size([1, 1, 20, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 21, 50]) torch.Size([1, 1, 18, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 41, 50]) torch.Size([1, 1, 38, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 22, 50]) torch.Size([1, 1, 19, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 8, 50]) torch.Size([1, 1, 5, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 50]) torch.Size([1, 1, 7, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 26, 50]) torch.Size([1, 1, 23, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 16, 50]) torch.Size([1, 1, 13, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 12, 50]) torch.Si

KeyboardInterrupt: 

In [94]:
while epoch < 700:
    
    total_loss = 0
    for i in range(len(trainLabels)):
        cnnOptimizer.zero_grad()
        old_hidden = hidden
        log_probs = cnnLayer(trainRawheadlines[i*25:(i+1)*25])
        loss = criterion(log_probs, trainLabelVar[i])
        total_loss += loss
    total_loss.backward()
    cnnOptimizer.step()
    train_loss.append(total_loss.data)
    if ((epoch) % 1) == 0:
        print(train_log_string % (datetime.now(), epoch, i+1, len(trainheadlines), np.mean(train_loss)))
    epoch = epoch + 1

2018-05-31 02:58:46.498536 :: Epoch 206 :: Iter 1611 / 1611 :: train loss: 7010.7637
2018-05-31 02:59:20.271197 :: Epoch 207 :: Iter 1611 / 1611 :: train loss: 6982.3809
2018-05-31 03:00:04.605592 :: Epoch 208 :: Iter 1611 / 1611 :: train loss: 6954.2974
2018-05-31 03:00:42.338689 :: Epoch 209 :: Iter 1611 / 1611 :: train loss: 6926.4424
2018-05-31 03:01:16.321869 :: Epoch 210 :: Iter 1611 / 1611 :: train loss: 6898.8496
2018-05-31 03:01:51.332262 :: Epoch 211 :: Iter 1611 / 1611 :: train loss: 6871.5366
2018-05-31 03:02:25.123894 :: Epoch 212 :: Iter 1611 / 1611 :: train loss: 6844.4893
2018-05-31 03:03:04.304170 :: Epoch 213 :: Iter 1611 / 1611 :: train loss: 6817.6558
2018-05-31 03:03:41.708142 :: Epoch 214 :: Iter 1611 / 1611 :: train loss: 6791.0835
2018-05-31 03:04:15.352229 :: Epoch 215 :: Iter 1611 / 1611 :: train loss: 6764.7769
2018-05-31 03:04:50.426450 :: Epoch 216 :: Iter 1611 / 1611 :: train loss: 6738.6885
2018-05-31 03:05:24.263958 :: Epoch 217 :: Iter 1611 / 1611 :: tr

2018-05-31 03:55:25.729455 :: Epoch 303 :: Iter 1611 / 1611 :: train loss: 5125.9067
2018-05-31 03:56:00.947299 :: Epoch 304 :: Iter 1611 / 1611 :: train loss: 5112.7021
2018-05-31 03:56:36.014548 :: Epoch 305 :: Iter 1611 / 1611 :: train loss: 5099.5986
2018-05-31 03:57:11.363041 :: Epoch 306 :: Iter 1611 / 1611 :: train loss: 5086.5601
2018-05-31 03:57:45.720338 :: Epoch 307 :: Iter 1611 / 1611 :: train loss: 5073.6113
2018-05-31 03:58:20.770634 :: Epoch 308 :: Iter 1611 / 1611 :: train loss: 5060.7593
2018-05-31 03:58:55.589541 :: Epoch 309 :: Iter 1611 / 1611 :: train loss: 5047.9658
2018-05-31 03:59:30.598945 :: Epoch 310 :: Iter 1611 / 1611 :: train loss: 5035.2788
2018-05-31 04:00:06.882961 :: Epoch 311 :: Iter 1611 / 1611 :: train loss: 5022.6572
2018-05-31 04:00:41.804596 :: Epoch 312 :: Iter 1611 / 1611 :: train loss: 5010.1138
2018-05-31 04:01:16.857882 :: Epoch 313 :: Iter 1611 / 1611 :: train loss: 4997.6436
2018-05-31 04:01:52.028852 :: Epoch 314 :: Iter 1611 / 1611 :: tr

2018-05-31 04:52:33.413637 :: Epoch 400 :: Iter 1611 / 1611 :: train loss: 4150.8174
2018-05-31 04:53:08.596294 :: Epoch 401 :: Iter 1611 / 1611 :: train loss: 4143.2241
2018-05-31 04:53:44.420517 :: Epoch 402 :: Iter 1611 / 1611 :: train loss: 4135.6406
2018-05-31 04:54:20.286338 :: Epoch 403 :: Iter 1611 / 1611 :: train loss: 4128.1509
2018-05-31 04:54:55.643811 :: Epoch 404 :: Iter 1611 / 1611 :: train loss: 4120.6572
2018-05-31 04:55:31.430134 :: Epoch 405 :: Iter 1611 / 1611 :: train loss: 4113.1812
2018-05-31 04:56:07.336141 :: Epoch 406 :: Iter 1611 / 1611 :: train loss: 4105.7651
2018-05-31 04:56:42.852188 :: Epoch 407 :: Iter 1611 / 1611 :: train loss: 4098.4004
2018-05-31 04:57:18.422092 :: Epoch 408 :: Iter 1611 / 1611 :: train loss: 4091.0325
2018-05-31 04:57:54.101700 :: Epoch 409 :: Iter 1611 / 1611 :: train loss: 4083.7031
2018-05-31 04:58:30.119408 :: Epoch 410 :: Iter 1611 / 1611 :: train loss: 4076.4265
2018-05-31 04:59:06.255795 :: Epoch 411 :: Iter 1611 / 1611 :: tr

2018-05-31 05:51:15.942353 :: Epoch 497 :: Iter 1611 / 1611 :: train loss: 3554.3091
2018-05-31 05:51:54.550771 :: Epoch 498 :: Iter 1611 / 1611 :: train loss: 3549.3657
2018-05-31 05:52:32.184922 :: Epoch 499 :: Iter 1611 / 1611 :: train loss: 3544.4260
2018-05-31 05:53:10.422097 :: Epoch 500 :: Iter 1611 / 1611 :: train loss: 3539.5085
2018-05-31 05:53:47.553749 :: Epoch 501 :: Iter 1611 / 1611 :: train loss: 3534.6038
2018-05-31 05:54:24.914676 :: Epoch 502 :: Iter 1611 / 1611 :: train loss: 3529.7385
2018-05-31 05:55:02.185536 :: Epoch 503 :: Iter 1611 / 1611 :: train loss: 3524.8923
2018-05-31 05:55:39.889856 :: Epoch 504 :: Iter 1611 / 1611 :: train loss: 3520.0493
2018-05-31 05:56:17.409853 :: Epoch 505 :: Iter 1611 / 1611 :: train loss: 3515.2485
2018-05-31 05:56:54.419097 :: Epoch 506 :: Iter 1611 / 1611 :: train loss: 3510.4470
2018-05-31 05:57:32.423533 :: Epoch 507 :: Iter 1611 / 1611 :: train loss: 3505.6711
2018-05-31 05:58:10.254113 :: Epoch 508 :: Iter 1611 / 1611 :: tr

2018-05-31 06:52:49.565721 :: Epoch 594 :: Iter 1611 / 1611 :: train loss: 3151.1890
2018-05-31 06:53:27.551119 :: Epoch 595 :: Iter 1611 / 1611 :: train loss: 3147.7180
2018-05-31 06:54:05.602736 :: Epoch 596 :: Iter 1611 / 1611 :: train loss: 3144.2471
2018-05-31 06:54:43.708654 :: Epoch 597 :: Iter 1611 / 1611 :: train loss: 3140.7922
2018-05-31 06:55:21.749420 :: Epoch 598 :: Iter 1611 / 1611 :: train loss: 3137.3586
2018-05-31 06:55:59.608901 :: Epoch 599 :: Iter 1611 / 1611 :: train loss: 3133.9146
2018-05-31 06:56:37.738900 :: Epoch 600 :: Iter 1611 / 1611 :: train loss: 3130.4978
2018-05-31 06:57:15.432906 :: Epoch 601 :: Iter 1611 / 1611 :: train loss: 3127.0842
2018-05-31 06:57:52.748563 :: Epoch 602 :: Iter 1611 / 1611 :: train loss: 3123.6853
2018-05-31 06:58:31.121754 :: Epoch 603 :: Iter 1611 / 1611 :: train loss: 3120.3049
2018-05-31 06:59:08.899633 :: Epoch 604 :: Iter 1611 / 1611 :: train loss: 3116.9170
2018-05-31 06:59:52.996154 :: Epoch 605 :: Iter 1611 / 1611 :: tr

2018-05-31 07:54:34.078528 :: Epoch 691 :: Iter 1611 / 1611 :: train loss: 2860.2146
2018-05-31 07:55:11.225660 :: Epoch 692 :: Iter 1611 / 1611 :: train loss: 2857.6304
2018-05-31 07:55:49.514784 :: Epoch 693 :: Iter 1611 / 1611 :: train loss: 2855.0574
2018-05-31 07:56:27.211013 :: Epoch 694 :: Iter 1611 / 1611 :: train loss: 2852.4792
2018-05-31 07:57:04.527840 :: Epoch 695 :: Iter 1611 / 1611 :: train loss: 2849.9153
2018-05-31 07:57:42.517051 :: Epoch 696 :: Iter 1611 / 1611 :: train loss: 2847.3518
2018-05-31 07:58:20.588927 :: Epoch 697 :: Iter 1611 / 1611 :: train loss: 2844.8015
2018-05-31 07:58:58.310989 :: Epoch 698 :: Iter 1611 / 1611 :: train loss: 2842.2678
2018-05-31 07:59:36.213815 :: Epoch 699 :: Iter 1611 / 1611 :: train loss: 2839.7312


In [98]:
while epoch < 900:
    
    total_loss = 0
    for i in range(len(trainLabels)):
        cnnOptimizer.zero_grad()
        old_hidden = hidden
        log_probs = cnnLayer(trainRawheadlines[i*25:(i+1)*25])
        loss = criterion(log_probs, trainLabelVar[i])
        total_loss += loss
    total_loss.backward()
    cnnOptimizer.step()
    train_loss.append(total_loss.data)
    if ((epoch) % 1) == 0:
        print(train_log_string % (datetime.now(), epoch, i+1, len(trainheadlines), np.mean(train_loss)))
    if ((epoch) % 30) == 0:
        trainPredictions = []
        log_probs = []
        h = hidden
        c = cell
        for i in range(len(trainLabels)):
            log_prob = cnnLayer(trainRawheadlines[i*25:(i+1)*25])
            log_probs.append(log_prob)
            if float(log_prob[0][0]) > float(log_prob[0][1]):
                trainPredictions.append(0)
            else:
                trainPredictions.append(1)
        print('Training Accuracy:', sum(np.asarray(trainPredictions) == np.asarray(trainLabels))/len(trainLabels))
        from sklearn.metrics import confusion_matrix
        print(confusion_matrix(trainLabels, trainPredictions))
    epoch = epoch + 1

2018-05-31 09:54:29.859436 :: Epoch 711 :: Iter 1611 / 1611 :: train loss: 2809.9766
2018-05-31 09:55:23.806349 :: Epoch 712 :: Iter 1611 / 1611 :: train loss: 2807.5449
2018-05-31 09:56:27.026627 :: Epoch 713 :: Iter 1611 / 1611 :: train loss: 2805.1277
2018-05-31 09:57:30.885067 :: Epoch 714 :: Iter 1611 / 1611 :: train loss: 2802.7197
2018-05-31 09:58:24.217944 :: Epoch 715 :: Iter 1611 / 1611 :: train loss: 2800.2988
2018-05-31 09:59:19.088781 :: Epoch 716 :: Iter 1611 / 1611 :: train loss: 2797.8770
2018-05-31 10:00:15.984260 :: Epoch 717 :: Iter 1611 / 1611 :: train loss: 2795.4802
2018-05-31 10:01:14.226009 :: Epoch 718 :: Iter 1611 / 1611 :: train loss: 2793.0674
2018-05-31 10:02:11.283112 :: Epoch 719 :: Iter 1611 / 1611 :: train loss: 2790.6714
2018-05-31 10:03:06.881890 :: Epoch 720 :: Iter 1611 / 1611 :: train loss: 2788.3108
Training Accuracy: 0.579143389199
[[227 511]
 [167 706]]
2018-05-31 10:04:44.481859 :: Epoch 721 :: Iter 1611 / 1611 :: train loss: 2785.9360
2018-05-

2018-05-31 11:23:28.608335 :: Epoch 806 :: Iter 1611 / 1611 :: train loss: 2604.8101
2018-05-31 11:24:21.441235 :: Epoch 807 :: Iter 1611 / 1611 :: train loss: 2602.8943
2018-05-31 11:25:16.529405 :: Epoch 808 :: Iter 1611 / 1611 :: train loss: 2600.9817
2018-05-31 11:26:07.957535 :: Epoch 809 :: Iter 1611 / 1611 :: train loss: 2599.0796
2018-05-31 11:27:01.380722 :: Epoch 810 :: Iter 1611 / 1611 :: train loss: 2597.2065
Training Accuracy: 0.606455617629
[[234 504]
 [130 743]]
2018-05-31 11:28:36.749945 :: Epoch 811 :: Iter 1611 / 1611 :: train loss: 2595.3203
2018-05-31 11:29:30.751970 :: Epoch 812 :: Iter 1611 / 1611 :: train loss: 2593.4385
2018-05-31 11:30:23.171056 :: Epoch 813 :: Iter 1611 / 1611 :: train loss: 2591.5571
2018-05-31 11:31:18.002827 :: Epoch 814 :: Iter 1611 / 1611 :: train loss: 2589.6958
2018-05-31 11:32:10.409441 :: Epoch 815 :: Iter 1611 / 1611 :: train loss: 2587.8389
2018-05-31 11:33:02.476194 :: Epoch 816 :: Iter 1611 / 1611 :: train loss: 2585.9768
2018-05-

In [100]:
state_dict = {
    'epoch': epoch,
    'cnnLayer': cnnLayer.state_dict(),
    'cnnOptimizer': cnnOptimizer.state_dict()
}
ckpt_path = 'data\ckpt\basic_most\model.pt'
# torch.save(state_dict, '/data/ckpt/basic_most/model.pt')
torch.save(state_dict, 'cnn_model_hidden_epoch900_mindif3_emb50_modified.pt')

In [162]:
# del lstmLayer
# del lstmOptimizer
# del train_loss
lstmLayer = LSTMLayer(EMBEDDING_SIZE, 100).cuda()
lstmOptimizer = torch.optim.Adam(lstmLayer.parameters(), lr=0.3)
ckpt = torch.load('bow_log_model_full_epoch200_mindf_3_emb300.pt')
lstmLayer.load_state_dict(ckpt['lstmLayer'])
lstmOptimizer.load_state_dict(ckpt['lstm_optimizer'])

In [96]:
trainPredictions = []
log_probs = []
h = hidden
c = cell
for i in range(len(trainLabels)):
#     log_prob = logisticLayer(trainheadlines[i])
#     if i%10 == 0:
#         print(i)
    log_prob = cnnLayer(trainRawheadlines[i*25:(i+1)*25])
    log_probs.append(log_prob)
    if float(log_prob[0][0]) > float(log_prob[0][1]):
        trainPredictions.append(0)
    else:
        trainPredictions.append(1)

In [97]:
print('Training Accuracy:', sum(np.asarray(trainPredictions) == np.asarray(trainLabels))/len(trainLabels))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(trainLabels, trainPredictions))

Training Accuracy: 0.592799503414
[[206 532]
 [124 749]]


In [101]:

testPredictions = []
log_probs = []
h = None
c = None
for i in range(len(testLabels)):
    log_prob = cnnLayer(testRawheadlines[i*25:(i+1)*25])
    log_probs.append(log_prob[0])
    if float(log_prob[0][0]) > float(log_prob[0][1]):
        testPredictions.append(0)
    else:
        testPredictions.append(1)

In [102]:
print('Test Accuracy:', sum(np.asarray(testPredictions) == np.asarray(testLabels))/len(testLabels))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(testLabels, testPredictions))

Test Accuracy: 0.484126984127
[[ 42 144]
 [ 51 141]]


In [152]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf

class LSTMLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LSTMLayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.lstmLayers = []
        for i in range(25):
            self.lstmLayers.append(nn.LSTM(input_dim, output_dim))
        self.basicLSTM = nn.LSTM(input_dim, output_dim)
        self.fc = nn.Linear(output_dim, 2)

    def forward(self, headlines, hidden = None, cell = None):
        outputs = []
        new_hiddens = []
        if hidden is None:
            hidden = Variable(torch.zeros(1, 1, self.output_dim)).float()
            cell = Variable(torch.zeros(1, 1, self.output_dim)).float()
        hidden = hidden.cuda()
        cell = cell.cuda()
        
        words = basicvectorizer.inverse_transform(headlines)
        embeddingSum = [0]*self.input_dim
        for word in words[0]:
            if word in embeddings_index:
                embeddingSum += embeddings_index[word]
        embeddingSumVar = Variable(torch.from_numpy(np.asarray(embeddingSum))).view(1, 1, -1).float().cuda()
#         print(embeddingSum)
        output, (hidden, cell) = self.basicLSTM(embeddingSumVar, (hidden, cell))
        log_probs = F.log_softmax(self.fc(output[0]), dim=-1)
        return log_probs, hidden, cell


In [227]:
# LSTM Training

lstmLayer = LSTMLayer(EMBEDDING_SIZE, 30).cuda()

learning_rate = 0.3
lstmOptimizer = torch.optim.Adam(lstmLayer.parameters(), lr=learning_rate)

epoch = 0
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
train_loss = []
criterion = torch.nn.NLLLoss()
trainLabelVar = Variable(torch.LongTensor(trainLabels)).cuda()

while epoch < 5:
    cell = None
    hidden = None
    total_loss = 0
    for (i, headlines) in enumerate(basictrain):
        if i == 500:
            break
        lstmOptimizer.zero_grad()
        log_probs, hidden, cell = lstmLayer(headlines, hidden, cell)
        loss = criterion(log_probs, trainLabelVar[i])
        total_loss += loss
    total_loss.backward()
        
    lstmOptimizer.step()
    train_loss.append(total_loss.data)
    if ((epoch) % 10) == 0:
        print(train_log_string % (datetime.now(), epoch, i+1, len(trainheadlines), np.mean(train_loss)), end='\n')
    epoch = epoch + 1

2018-05-16 11:12:23.439718 :: Epoch 0 :: Iter 501 / 1611 :: train loss: 347.2811


In [216]:
# lstmLayer.cpu()
trainPredictions = []
log_probs = []
h = hidden
c = cell
for i in range(len(trainLabels)):
    log_prob, h, c = lstmLayer(basictrain[i], h, c)
    log_probs.append(log_prob)
    if float(log_prob[0][0]) > float(log_prob[0][1]):
        trainPredictions.append(0)
    else:
        trainPredictions.append(1)

In [ ]:

testPredictions = []
log_probs = []
h = None
c = None
for i in range(len(testLabels)):
    log_prob,h,c = lstmLayer(basictest[i],h,c)
    log_probs.append(log_prob[0])
    if float(log_prob[0][0]) > float(log_prob[0][1]):
        testPredictions.append(0)
    else:
        testPredictions.append(1)